# Classificando letras em determinadas fontes

## Extração dos CSVs do arquivo .zip

In [ ]:
with zipfile.ZipFile("fonts.zip") as fonts_zip:
    csv_files = fonts_zip.namelist()
    x = pd.DataFrame()
    fonts_list = {}
    for csv in csv_files:
        font = fonts_zip.open(csv)
        size = font.read()
        fonts_list[font] = sys.getsizeof(size)

Neste trecho, ordenamos os arquivos por quantidade de linhas de maneira decrescente. Utilizamos para a análise do problema, 5 arquivos, cada um contendo letras de um determinado tipo de fonte.



In [ ]:
        sorted_list = sorted(fonts_list.items(), key=operator.itemgetter(1), reverse=True)
        chosen_ones = sorted_list[:5]
        fonts_list = []

Após ordenar, pegamos o nome de todas as fontes escolhidas e lemos 5000 linhas de seus respectivos CSVs.

In [ ]:
    for font in chosen_ones:
        font = fonts_zip.open(font[0].name)
        df = pd.read_csv(font, index_col=None, header=0, nrows=5000)
        fonts_list.append(df)
    x = pd.concat(fonts_list)

## Construção da solução

Neste ponto, armazenamos a coluna 'fonts', que contem os nomes das fontes em y_aux, ou seja, suas <i><b>labels</b></i> e retiramos as colunas que são 
irrelevantes para nossa solução.

In [ ]:
y_aux = x[['font']]
x.drop("h", axis=1, inplace=True)
x.drop("w", axis=1, inplace=True)
x.drop('font', axis=1, inplace=True)
x.drop('fontVariant', axis=1, inplace=True)


Em seguida trasnformamos as <i><b>labels</b></i> armarzenadas em inteiros e  separamos 30% de cada CSVs para treinamento do algoritmo. 

In [ ]:
y = y_aux.apply(LabelEncoder().fit_transform)
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=42)


É após esta divisão que escolhemos o algoritmo utilizado para predição e 
executamos a mesma. Abaixo estarão exemplos de código utilizando a 
<i><b>Decision Tree</b></i> e <i><b>KNN</b></i>.

### Decision Tree

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(x_treino, y_treino)
predictions = clf.predict(x_teste)
print(accuracy_score(y_teste, predictions))

### KNN

In [ ]:
for i in range(4, 8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_treino, y_treino.values.ravel())
    print("Knn with " + str(i) + " neighbors")
print(knn.score(x_teste, y_teste))